In [1]:
import glob
import imageio
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
from tensorflow.keras import layers
import time
from tensorflow.keras import Model
import tensorflow as tf
import tensorflow_addons as tfa

from IPython import display

## Attention Layer

### Ref Code

#### Compact v1 format

In [3]:
def contract_inner(x, y):
#   """tensordot(x, y, 1)."""
  x_chars = list(string.ascii_lowercase[:len(x.shape)])
  y_chars = list(string.ascii_uppercase[:len(y.shape)])
  assert len(x_chars) == len(x.shape) and len(y_chars) == len(y.shape)
  y_chars[0] = x_chars[-1]  # first axis of y and last of x get summed
  out_chars = x_chars[:-1] + y_chars[1:]
  return _einsum(x_chars, y_chars, out_chars, x, y)


##### Replace with tf.tensordot

#### Testing tf.tensorfot io behavior

In [9]:
x = tf.constant(2, shape=(2,32,32,3))
w = tf.constant(1, shape=(3,3))
print(x.shape, w.shape)

(2, 32, 32, 3) (3, 3)


In [10]:
y = tf.tensordot(x, w, [[0],[-1]])

InvalidArgumentError: Matrix size-incompatible: In[0]: [3072,2], In[1]: [3,3] [Op:MatMul]

In [ ]:
def nin(x, *, name, num_units, init_scale=1.):
  with tf.variable_scope(name):
    in_dim = int(x.shape[-1])
    W = tf.get_variable('W', shape=[in_dim, num_units], initializer=default_init(scale=init_scale), dtype=DEFAULT_DTYPE)
    b = tf.get_variable('b', shape=[num_units], initializer=tf.constant_initializer(0.), dtype=DEFAULT_DTYPE)
    y = contract_inner(x, W) + b
    assert y.shape == x.shape[:-1] + [num_units]
    return y


In [2]:
def attn_block(x, *, name, temb):
  B, H, W, C = x.shape
  with tf.variable_scope(name):
    h = normalize(x, temb=temb, name='norm')
    q = nin(h, name='q', num_units=C)
    k = nin(h, name='k', num_units=C)
    v = nin(h, name='v', num_units=C)

    w = tf.einsum('bhwc,bHWc->bhwHW', q, k) * (int(C) ** (-0.5))
    w = tf.reshape(w, [B, H, W, H * W])
    w = tf.nn.softmax(w, -1)
    w = tf.reshape(w, [B, H, W, H, W])

    h = tf.einsum('bhwHW,bHWc->bhwc', w, v)
    h = nn.nin(h, name='proj_out', num_units=C, init_scale=0.)

    assert h.shape == x.shape
    print(tf.get_default_graph().get_name_scope(), x.shape)
    return x + h